In [424]:
# coding: utf-8
import os, sys
sys.path.append(os.pardir)
import numpy as np
from common.functions import *

class Relu():
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout
        return dx

class Sigmoid():
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1/(1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1 - self.out)
        return dx
    
class Affine():
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        dx = np.dot(dout, self.W.T)
        dx = dx.reshape(*self.original_x_shape)
        return dx
    
    
class SoftmaxWithLoss():
    def __init__(self):
        self.x = None
        self.y = None
        self.t = None
        self.loss = None

    def forward(self, x, t):
        self.y = softmax(x)
        self.t = t
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.t - self.y) / batch_size
        return dx
    

class DropOut():
    def __init__(self, dropout_ratio):
        self.mask = None
        self.dropout_ratio = dropout_ratio
        
    def forward(self, x):
        self.mask = np.random.randn(*x.shape) >= self.dropout_ratio
        return x * self.mask
    
    def backward(self, dout):
        return dout * self.mask
    

class Convolution():
    def __init__(self, W, b, stride, pad):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.col = None
        self.col_W = None
        
        self.dW = None
        self.db = None
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape 
        N, C, H, W = self.x.shape
        
        out_h = int((H + 2 * pad - FH)/stride) + 1
        out_w = int((W + 2 *pad - FW)/stride ) + 1
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_w = self.W.reshape(FN, -1).T
        
        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_W, -1).transpose(0, 3, 1, 2)
        self.x = x
        self.col = col
        self.col_W = col_W    
        return out
    
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,3,1,2).reshape(-1,FN)
        
        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)
        
        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)
        
        return dx
    
    
    class Pooling():
        def __init__(self, pool_h, pool_w, stride=1, pad=0):
            self.pool_h = pool_h
            self.pool_w = pool_w
            self.stride = stride
            self.pad = pad
            
        def forward(self, x):
            N, C, H, W = x.shape
            out_h = int((H - self.pool_h)/stride + 1)
            out_h = int((W - self.pool_w)/stride + 1)
            
            col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
            col = col.reshape(-1, self.pool_h * self.pool_w)
            
            arg_max = np.argmax(col, axis=1)
            out = np.max(col, axis=1)
            out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
            
            self.x = x
            self.arg_max = arg_max
            return out
        
        def backward(self, dout):
            dout = dout.transpose(0, 2, 3, 1)
            pool_size = self.pool_h * self.pool_w
            dmax = np.zeros((dout.size, pool_size))
            dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
            dmax = dmax.reshape(dout.shape + (pool_size,)) 

            dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
            dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)

            return dx

            
    
        


In [432]:
x[mask]

array([1, 2, 3, 4, 5])

In [433]:
x * mask

array([[1, 2, 3],
       [4, 5, 0]])

In [465]:
np.random.randn(2,3)

array([[-0.14174202, -0.62967909,  0.68546723],
       [-0.9574383 , -1.86532133, -0.15938614]])

In [476]:
np.sum(np.random.rand(100,3) > 0.5)

148